In [ ]:
#https://www.metacritic.com/browse/movies/score/metascore/all/filtered

In [448]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [467]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}
response = requests.get('https://www.metacritic.com/browse/movies/score/metascore/all/filtered', headers = headers)
doc = BeautifulSoup(response.text)
doc

<!DOCTYPE html>

<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head><script type="text/javascript">
//Copyright Instart Fri May 24 2019 - All rights reserved - version: 10.6.32
function Ha(){var t=window;if(function(){var n=!0,i=!1,e="";try{var r="object"==typeof t.IXC_144_5393438182331658&&t.IXC_144_5393438182331658;if(n=!r||void 0===r.CanRun||"_144_5393438182331658"!==r._144_5393438182331658||r.CanRun("abd"))r=t.IXC_144_5393438182331658=t.IXC_144_5393438182331658||{},r.InitStartTime=(t.performance?t.performance:Date).now(),r._144_5393438182331658="_144_5393438182331658",t.INSTART_TARGET_NAME="abd",t.I12C=t.I12C||{};else{var o=t.INSTART_TARGET_NAME;e="double nanovisor injection: abd after "+o,(i="abd"!==o)&&console.warn(e)}t.INSTART=t.INSTART||{},t.INSTART.Init=function(){i&&console.error("ignored Init call after ",e)},r&&(r.ComputeAll="CENOME_PLACEHOLDER")}catch(t){try{console.error(t)}catch(t){}n=!0}return n}()){!function(){function n(){retu

In [468]:
rows = []
tables = doc.find_all('table')
for table in tables:
    movies = table.find_all('tr', class_ = False)
    for movie in movies:
        row = {}
        row['title'] = movie.h3.text
        row['release_date'] = movie.find(class_ = 'clamp-details').find('span', class_ = False).text
        row['meta_score'] = movie.find(class_ = 'clamp-score-wrap').text.strip()
        row['user_score'] = movie.find(class_ = 'clamp-userscore').a.text.strip()
        row['rank'] = movie.find(class_ = 'title numbered').text.strip().replace('.', '')
        rating = movie.find(class_ = 'clamp-details').find('br').find_next_sibling()
        if 'span' in str(rating):
            row['rated'] = ' '.join(rating['class'][1:]).strip()
        elif 'img' in str(rating):
            row['rated'] = re.findall(r'.+mpaa/(.+)\.svg' ,rating.get_attribute_list('src')[0])[0]
        row['url'] = movie.find(class_ = 'clamp-summary-wrap').a['href']
        row['img'] = movie.a['href']
        row['summary'] = movie.find(class_ = 'summary').text.strip()
        rows.append(row)


In [469]:
df = pd.DataFrame(rows)
df.head()

,img,meta_score,rank,rated,release_date,summary,title,url,user_score
0,/movie/citizen-kane,100,1,Approved,"September 4, 1941","Following the death of a publishing tycoon, ne...",Citizen Kane,/movie/citizen-kane/critic-reviews,8.4
1,/movie/the-godfather,100,2,RATED_R,"March 11, 1972",Francis Ford Coppola's epic features Marlon Br...,The Godfather,/movie/the-godfather/critic-reviews,9.2
2,/movie/rear-window,100,3,TV-G,"September 1, 1954",A wheelchair-bound photographer spies on his n...,Rear Window,/movie/rear-window/critic-reviews,8.8
3,/movie/casablanca,100,4,TV-PG,"January 23, 1943","A Casablanca, Morocco casino owner in 1941 she...",Casablanca,/movie/casablanca/critic-reviews,9.0
4,/movie/boyhood,100,5,RATED_R,"July 11, 2014","Filmed over 12 years with the same cast, Richa...",Boyhood,/movie/boyhood/critic-reviews,7.7


In [470]:
df = df[['title', 'rank', 'url', 'meta_score', 'user_score', 'release_date', 'rated', 'summary', 'img']]
df.head()

,title,rank,url,meta_score,user_score,release_date,rated,summary,img
0,Citizen Kane,1,/movie/citizen-kane/critic-reviews,100,8.4,"September 4, 1941",Approved,"Following the death of a publishing tycoon, ne...",/movie/citizen-kane
1,The Godfather,2,/movie/the-godfather/critic-reviews,100,9.2,"March 11, 1972",RATED_R,Francis Ford Coppola's epic features Marlon Br...,/movie/the-godfather
2,Rear Window,3,/movie/rear-window/critic-reviews,100,8.8,"September 1, 1954",TV-G,A wheelchair-bound photographer spies on his n...,/movie/rear-window
3,Casablanca,4,/movie/casablanca/critic-reviews,100,9.0,"January 23, 1943",TV-PG,"A Casablanca, Morocco casino owner in 1941 she...",/movie/casablanca
4,Boyhood,5,/movie/boyhood/critic-reviews,100,7.7,"July 11, 2014",RATED_R,"Filmed over 12 years with the same cast, Richa...",/movie/boyhood


In [472]:
# df['distribution'] = 
import numpy as np
distribution = df['summary'].str.extract('.+\. \[(.+)\].*|.+\. \((.+)\).*')
distribution = distribution[0].fillna('').astype(str) + distribution[1].fillna('').astype(str)
df['distribution'] = distribution.apply(lambda x: np.nan if x=='' else x)
df

,title,rank,url,meta_score,user_score,release_date,rated,summary,img,distribution
0,Citizen Kane,1,/movie/citizen-kane/critic-reviews,100,8.4,"September 4, 1941",Approved,"Following the death of a publishing tycoon, ne...",/movie/citizen-kane,NaN
1,The Godfather,2,/movie/the-godfather/critic-reviews,100,9.2,"March 11, 1972",RATED_R,Francis Ford Coppola's epic features Marlon Br...,/movie/the-godfather,Paramount Pictures
2,Rear Window,3,/movie/rear-window/critic-reviews,100,8.8,"September 1, 1954",TV-G,A wheelchair-bound photographer spies on his n...,/movie/rear-window,NaN
3,Casablanca,4,/movie/casablanca/critic-reviews,100,9.0,"January 23, 1943",TV-PG,"A Casablanca, Morocco casino owner in 1941 she...",/movie/casablanca,NaN
4,Boyhood,5,/movie/boyhood/critic-reviews,100,7.7,"July 11, 2014",RATED_R,"Filmed over 12 years with the same cast, Richa...",/movie/boyhood,IFC Films
5,Three Colors: Red,6,/movie/three-colors-red/critic-reviews,100,8.8,"November 23, 1994",RATED_R,Krzysztof Kieslowski closes his Three Colors t...,/movie/three-colors-red,Criterion
6,Vertigo,7,/movie/vertigo/critic-reviews,100,8.7,"May 28, 1958",RATED_PG,Vertigo creates a dizzying web of mistaken ide...,/movie/vertigo,Universal Pictures
7,Notorious,8,/movie/notorious-1946/critic-reviews,100,7.7,"September 6, 1946",Approved,A woman is asked to spy on a group of Nazi fri...,/movie/notorious-1946,NaN
8,Singin' in the Rain,9,/movie/singin-in-the-rain/critic-reviews,99,8.9,"April 11, 1952",RATED_G,A silent film production company and cast make...,/movie/singin-in-the-rain,NaN
9,Moonlight,10,/movie/moonlight-2016/critic-reviews,99,7.2,"October 21, 2016",Not Rated,"Moonlight is the tender, heartbreaking story o...",/movie/moonlight-2016,NaN


In [474]:
df['summary'] = df.summary.str.replace('\[.+\]', '').str.replace(' \(.+\)', '')
df.head()

,title,rank,url,meta_score,user_score,release_date,rated,summary,img,distribution
0,Citizen Kane,1,/movie/citizen-kane/critic-reviews,100,8.4,"September 4, 1941",Approved,"Following the death of a publishing tycoon, ne...",/movie/citizen-kane,NaN
1,The Godfather,2,/movie/the-godfather/critic-reviews,100,9.2,"March 11, 1972",RATED_R,Francis Ford Coppola's epic features Marlon Br...,/movie/the-godfather,Paramount Pictures
2,Rear Window,3,/movie/rear-window/critic-reviews,100,8.8,"September 1, 1954",TV-G,A wheelchair-bound photographer spies on his n...,/movie/rear-window,NaN
3,Casablanca,4,/movie/casablanca/critic-reviews,100,9.0,"January 23, 1943",TV-PG,"A Casablanca, Morocco casino owner in 1941 she...",/movie/casablanca,NaN
4,Boyhood,5,/movie/boyhood/critic-reviews,100,7.7,"July 11, 2014",RATED_R,"Filmed over 12 years with the same cast, Richa...",/movie/boyhood,IFC Films


In [447]:
df.to_csv('metacrtic.csv', index = False)

In [ ]:
# https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page={}

In [461]:
rows = []
for num in range(1, 6):
    url = f'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page={num}'
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}
    response = requests.get(url, headers = headers)
    doc = BeautifulSoup(response.text)
    tables = doc.find_all('table')
    print(url)
    print(doc)
    for table in tables:
        movies = table.find_all('tr', class_ = False)
        for movie in movies:
            row = {}
            row['title'] = movie.h3.text
            row['release_date'] = movie.find(class_ = 'clamp-details').find('span', class_ = False).text
            row['meta_score'] = movie.find(class_ = 'clamp-score-wrap').text.strip()
            row['user_score'] = movie.find(class_ = 'clamp-userscore').a.text.strip()
            row['rank'] = movie.find(class_ = 'title numbered').text.strip().replace('.', '')
            rating = movie.find(class_ = 'clamp-details').find('br').find_next_sibling()
            if 'span' in str(rating):
                row['rated'] = ' '.join(rating['class'][1:]).strip()
            elif 'img' in str(rating):
                row['rated'] = re.findall(r'.+mpaa/(.+)\.svg' ,rating.get_attribute_list('src')[0])[0]
            row['url'] = movie.find(class_ = 'clamp-summary-wrap').a['href']
            row['img'] = movie.a['href']
            row['summary'] = movie.find(class_ = 'summary').text.strip()
            rows.append(row)




https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page=1
<!DOCTYPE html>

<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head><script type="text/javascript">
//Copyright Instart Fri May 24 2019 - All rights reserved - version: 10.6.32
function Ha(){var t=window;if(function(){var n=!0,i=!1,e="";try{var r="object"==typeof t.IXC_144_5393438182331658&&t.IXC_144_5393438182331658;if(n=!r||void 0===r.CanRun||"_144_5393438182331658"!==r._144_5393438182331658||r.CanRun("abd"))r=t.IXC_144_5393438182331658=t.IXC_144_5393438182331658||{},r.InitStartTime=(t.performance?t.performance:Date).now(),r._144_5393438182331658="_144_5393438182331658",t.INSTART_TARGET_NAME="abd",t.I12C=t.I12C||{};else{var o=t.INSTART_TARGET_NAME;e="double nanovisor injection: abd after "+o,(i="abd"!==o)&&console.warn(e)}t.INSTART=t.INSTART||{},t.INSTART.Init=function(){i&&console.error("ignored Init call after ",e)},r&&(r.ComputeAll="CENOME_PLACEHOLDER")}catch(t){t

https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page=2
<!DOCTYPE html>

<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head><script type="text/javascript">
//Copyright Instart Fri May 24 2019 - All rights reserved - version: 10.6.32
function Ha(){var t=window;if(function(){var n=!0,i=!1,e="";try{var r="object"==typeof t.IXC_144_5393438182331658&&t.IXC_144_5393438182331658;if(n=!r||void 0===r.CanRun||"_144_5393438182331658"!==r._144_5393438182331658||r.CanRun("abd"))r=t.IXC_144_5393438182331658=t.IXC_144_5393438182331658||{},r.InitStartTime=(t.performance?t.performance:Date).now(),r._144_5393438182331658="_144_5393438182331658",t.INSTART_TARGET_NAME="abd",t.I12C=t.I12C||{};else{var o=t.INSTART_TARGET_NAME;e="double nanovisor injection: abd after "+o,(i="abd"!==o)&&console.warn(e)}t.INSTART=t.INSTART||{},t.INSTART.Init=function(){i&&console.error("ignored Init call after ",e)},r&&(r.ComputeAll="CENOME_PLACEHOLDER")}catch(t){t

https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page=3
<!DOCTYPE html>

<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head><script type="text/javascript">
//Copyright Instart Fri May 24 2019 - All rights reserved - version: 10.6.32
function Ha(){var t=window;if(function(){var n=!0,i=!1,e="";try{var r="object"==typeof t.IXC_144_5393438182331658&&t.IXC_144_5393438182331658;if(n=!r||void 0===r.CanRun||"_144_5393438182331658"!==r._144_5393438182331658||r.CanRun("abd"))r=t.IXC_144_5393438182331658=t.IXC_144_5393438182331658||{},r.InitStartTime=(t.performance?t.performance:Date).now(),r._144_5393438182331658="_144_5393438182331658",t.INSTART_TARGET_NAME="abd",t.I12C=t.I12C||{};else{var o=t.INSTART_TARGET_NAME;e="double nanovisor injection: abd after "+o,(i="abd"!==o)&&console.warn(e)}t.INSTART=t.INSTART||{},t.INSTART.Init=function(){i&&console.error("ignored Init call after ",e)},r&&(r.ComputeAll="CENOME_PLACEHOLDER")}catch(t){t

https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page=4
<!DOCTYPE html>

<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head><script type="text/javascript">
//Copyright Instart Fri May 24 2019 - All rights reserved - version: 10.6.32
function Ha(){var t=window;if(function(){var n=!0,i=!1,e="";try{var r="object"==typeof t.IXC_144_5393438182331658&&t.IXC_144_5393438182331658;if(n=!r||void 0===r.CanRun||"_144_5393438182331658"!==r._144_5393438182331658||r.CanRun("abd"))r=t.IXC_144_5393438182331658=t.IXC_144_5393438182331658||{},r.InitStartTime=(t.performance?t.performance:Date).now(),r._144_5393438182331658="_144_5393438182331658",t.INSTART_TARGET_NAME="abd",t.I12C=t.I12C||{};else{var o=t.INSTART_TARGET_NAME;e="double nanovisor injection: abd after "+o,(i="abd"!==o)&&console.warn(e)}t.INSTART=t.INSTART||{},t.INSTART.Init=function(){i&&console.error("ignored Init call after ",e)},r&&(r.ComputeAll="CENOME_PLACEHOLDER")}catch(t){t

https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page=5
<!DOCTYPE html>

<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head><script type="text/javascript">
//Copyright Instart Fri May 24 2019 - All rights reserved - version: 10.6.32
function Ha(){var t=window;if(function(){var n=!0,i=!1,e="";try{var r="object"==typeof t.IXC_144_5393438182331658&&t.IXC_144_5393438182331658;if(n=!r||void 0===r.CanRun||"_144_5393438182331658"!==r._144_5393438182331658||r.CanRun("abd"))r=t.IXC_144_5393438182331658=t.IXC_144_5393438182331658||{},r.InitStartTime=(t.performance?t.performance:Date).now(),r._144_5393438182331658="_144_5393438182331658",t.INSTART_TARGET_NAME="abd",t.I12C=t.I12C||{};else{var o=t.INSTART_TARGET_NAME;e="double nanovisor injection: abd after "+o,(i="abd"!==o)&&console.warn(e)}t.INSTART=t.INSTART||{},t.INSTART.Init=function(){i&&console.error("ignored Init call after ",e)},r&&(r.ComputeAll="CENOME_PLACEHOLDER")}catch(t){t

In [465]:
df_101 = pd.DataFrame(rows)

In [475]:
df_101 = df_101[['title', 'rank', 'url', 'meta_score', 'user_score', 'release_date', 'rated', 'summary', 'img']]
df_101.head()

,title,rank,url,meta_score,user_score,release_date,rated,summary,img
0,The Decline of Western Civilization,101,/movie/the-decline-of-western-civilization/cri...,93,tbd,"July 5, 1981",Not Rated,The Los Angeles punk music scene circa 1980 is...,/movie/the-decline-of-western-civilization
1,Days of Heaven,102,/movie/days-of-heaven/critic-reviews,93,8.6,"September 13, 1978",RATED_PG,One of the most critically acclaimed films of ...,/movie/days-of-heaven
2,Crumb,103,/movie/crumb/critic-reviews,93,8.4,"April 28, 1995",RATED_R,A documentary of artist Robert Crumb's life.,/movie/crumb
3,The Band Wagon,104,/movie/the-band-wagon/critic-reviews,93,tbd,"August 7, 1953",Passed,A pretentiously artistic director is hired for...,/movie/the-band-wagon
4,One More Time with Feeling,105,/movie/one-more-time-with-feeling/critic-reviews,92,8.6,"September 8, 2016",Not Rated,A unique one night only cinema event directed ...,/movie/one-more-time-with-feeling


In [476]:
import numpy as np
distribution = df_101['summary'].str.extract('.+\. \[(.+)\].*|.+\. \((.+)\).*')
distribution = distribution[0].fillna('').astype(str) + distribution[1].fillna('').astype(str)
df_101['distribution'] = distribution.apply(lambda x: np.nan if x=='' else x)
df_101['summary'] = df_101.summary.str.replace('\[.+\]', '').str.replace(' \(.+\)', '')
df_101

,title,rank,url,meta_score,user_score,release_date,rated,summary,img,distribution
0,The Decline of Western Civilization,101,/movie/the-decline-of-western-civilization/cri...,93,tbd,"July 5, 1981",Not Rated,The Los Angeles punk music scene circa 1980 is...,/movie/the-decline-of-western-civilization,NaN
1,Days of Heaven,102,/movie/days-of-heaven/critic-reviews,93,8.6,"September 13, 1978",RATED_PG,One of the most critically acclaimed films of ...,/movie/days-of-heaven,NaN
2,Crumb,103,/movie/crumb/critic-reviews,93,8.4,"April 28, 1995",RATED_R,A documentary of artist Robert Crumb's life.,/movie/crumb,NaN
3,The Band Wagon,104,/movie/the-band-wagon/critic-reviews,93,tbd,"August 7, 1953",Passed,A pretentiously artistic director is hired for...,/movie/the-band-wagon,NaN
4,One More Time with Feeling,105,/movie/one-more-time-with-feeling/critic-reviews,92,8.6,"September 8, 2016",Not Rated,A unique one night only cinema event directed ...,/movie/one-more-time-with-feeling,NaN
5,The Diving Bell and the Butterfly,106,/movie/the-diving-bell-and-the-butterfly/criti...,92,8.3,"November 30, 2007",RATED_PG-13,The Diving Bell and the Butterfly is the remar...,/movie/the-diving-bell-and-the-butterfly,Miramax Film
6,Invasion of the Body Snatchers,107,/movie/invasion-of-the-body-snatchers-1956/cri...,92,8.8,"February 5, 1956",Approved,A small-town doctor learns that the population...,/movie/invasion-of-the-body-snatchers-1956,NaN
7,The Class,108,/movie/the-class/critic-reviews,92,7.4,"December 19, 2008",RATED_PG-13,François and his fellow teachers prepare for a...,/movie/the-class,Sony Classics
8,Platoon,109,/movie/platoon/critic-reviews,92,8.1,"February 6, 1987",RATED_R,A young recruit in Vietnam faces a moral crisi...,/movie/platoon,NaN
9,35 Shots of Rum,110,/movie/35-shots-of-rum/critic-reviews,92,6.3,"September 16, 2009",Not Rated,"A widowed conductor, looking forward to retire...",/movie/35-shots-of-rum,NaN


In [479]:
pd.concat([df, df_101]).to_csv('metacritic_top600.csv', index = False)